# EURO CUP2024 - GERMANY

## Table for the Combination

In [0]:
%fs 
rm -r dbfs:/user/hive/warehouse/matches


res4: Boolean = true

In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/thirdplacecombination

res5: Boolean = true

In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/teamsgroups 

res6: Boolean = true

In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/matchstats

res7: Boolean = true

In [0]:
%sql
CREATE or replace  TABLE ThirdPlaceCombination(
  ID INTEGER,
  SELECTEDGROUPS VARCHAR(4),
  Team1B CHAR(1),
  Team1C CHAR(1),
  Team1E CHAR(1),
  Team1F CHAR(1)
);

-- Step 2: Insert the provided values into the "THIRDMATCHUP" table
INSERT INTO  ThirdPlaceCombination (ID, SELECTEDGROUPS, Team1B, Team1C, Team1E, Team1F) VALUES
(1, 'ABCD', 'A', 'D', 'B', 'C'),
(2, 'ABCE', 'A', 'E', 'B', 'C'),
(3, 'ABCF', 'A', 'F', 'B', 'C'),
(4, 'ABDE', 'D', 'E', 'A', 'B'),
(5, 'ABDF', 'D', 'F', 'A', 'B'),
(6, 'ABEF', 'E', 'F', 'B', 'A'),
(7, 'ACDE', 'E', 'D', 'C', 'A'),
(8, 'ACDF', 'F', 'D', 'C', 'A'),
(9, 'ACEF', 'E', 'F', 'C', 'A'),
(10, 'ADEF', 'E', 'F', 'D', 'A'),
(11, 'BCDE', 'E', 'D', 'B', 'C'),
(12, 'BCDF', 'F', 'D', 'C', 'B'),
(13, 'BCEF', 'F', 'E', 'C', 'B'),
(14, 'BDEF', 'F', 'E', 'D', 'B'),
(15, 'CDEF', 'F', 'E', 'D', 'C');

num_affected_rows,num_inserted_rows
15,15


## Table for the Groups of Teams

In [0]:
%sql
CREATE or replace TABLE TeamsGroups (
    TeamID INT,
    TeamName VARCHAR(100),
    GroupName VARCHAR(1) 
);

INSERT INTO TeamsGroups (TeamID,TeamName,GroupName) VALUES
-- Group A
    (1, 'Germany','A'),
    (2, 'Scotland','A'),
    (3, 'Hungary','A'),
    (4, 'Switzerland','A'),

-- Group B
    (5, 'Spain','B'),
    (6, 'Croatia','B'),
    (7, 'Italy','B'),
    (8, 'Albania','B'),

-- Group C
    (9, 'Slovenia','C'),
    (10, 'Denmark','C'),
    (11, 'Serbia','C'),
    (12, 'England','C'),

-- Group D
    (13, 'Netherlands','D'),
    (14, 'Austria','D'),
    (15, 'France','D'),
    (16, 'Play-off Winner A','D'),

-- Group E
    (17, 'Belgium','E'),
    (18, 'Slovakia','E'),
    (19, 'Romania','E'),
    (20, 'Play-off Winner B','E'),

-- Group F
    (21, 'Turkey','F'),
    (22, 'Play-off Winner C','F'),
    (23, 'Portugal','F'),
    (24, 'Czech Republic','F');


num_affected_rows,num_inserted_rows
24,24


## Table for the ALL The matches which will be in the eurocup

In [0]:
%sql
CREATE or replace TABLE Matches (
    MatchID INT,
    Fixture varchar(50),
    GroupName varchar(1),
    HomeTeam varchar(20),
    AwayTeam varchar(20),
    HomeTeamGoals INT,
    AwayTeamGoals INT,
    MatchResult varchar(20),
    MatchStage varchar(20)
);

In [0]:
%sql
--Fixture of all teams from all the group and inserting into matches table
INSERT INTO MATCHES
    SELECT
        ROW_NUMBER() over(order by 1) as MatchId,
        CONCAT(T1.TeamName, ' vs ', T2.TeamName) AS Fixture,
        T1.GroupName AS GroupName,
        T1.TeamName as HomeTeam,
        T2.TeamName as AwayTeam,
        CAST(rand() * 6 AS INT) AS GoalsScored, 
        CAST(rand() * 6 AS INT) AS GoalsAllowed,
        'None' as MatchResult,
        'Group Stage' as MatchStage
    FROM 
        TeamsGroups AS T1
    CROSS JOIN 
        TeamsGroups AS T2 where T1.GroupName = T2.GroupName AND T1.TeamID < T2.TeamID;

num_affected_rows,num_inserted_rows
36,36


In [0]:
%sql
--Match result from the GroupMatchFixture
UPDATE Matches
SET MatchResult = 
    CASE
        WHEN HOMETEAMGOALS > AWAYTEAMGOALS THEN HomeTeam
        WHEN HOMETEAMGOALS < AWAYTEAMGOALS THEN AwayTeam
        ELSE 'Draw'
    END;

num_affected_rows
36


## Create a new table to store the group stage match statistic

In [0]:
%sql
--Match Statistics like win,loss,draw etc from the group stage matches
create or replace table MatchStats as 
    SELECT
        TeamID,
        GroupName,
         COUNT(*) AS MatchesPlayed,
        SUM(CASE 
            WHEN GoalsScored > GoalsAllowed THEN 1 
            ELSE 0 
        END) AS Wins,
        SUM(CASE 
            WHEN GoalsScored = GoalsAllowed THEN 1 
            ELSE 0 
        END) AS Draws,
        SUM(CASE 
            WHEN GoalsScored < GoalsAllowed THEN 1 
            ELSE 0 
        END) AS Losses,
        SUM(GoalsScored) AS GoalsScored,
        SUM(GoalsAllowed) AS GoalsAllowed,
        SUM(GoalsScored) - SUM(GoalsAllowed) AS GoalDifference,
        SUM(CASE
                WHEN GoalsScored > GoalsAllowed THEN 3  
                WHEN GoalsScored = GoalsAllowed THEN 1 
                ELSE 0                                
            END) AS Points
    FROM
        (SELECT
            GroupName,
            HomeTeam AS TeamID,
            HOMETEAMGOALS AS GoalsScored,
            AWAYTEAMGOALS AS GoalsAllowed
        FROM
            MATCHES
    
        UNION ALL
    
        SELECT
            GroupName,
            AwayTeam AS TeamID,
            AWAYTEAMGOALS AS GoalsScored,
            HOMETEAMGOALS AS GoalsAllowed
        FROM
            Matches) AS GroupMatches
    GROUP BY
        TeamID,groupname
        Order by groupname;

num_affected_rows,num_inserted_rows


## Create new table to store all the qualifiying teams from each group

In [0]:
%sql
--All the Teams who have quallified into round of 16

--Top 2 teams from each tables which are qualified for R16
create or replace table round_of_16 as 
SELECT *
FROM (
    SELECT 
        TeamId,
        GroupName,
        points,
        goaldifference,
        ROW_NUMBER() OVER (PARTITION BY GroupName ORDER BY points DESC, goaldifference DESC,goalsScored desc,wins desc) AS rnk 
    FROM 
        matchStats 
) AS Top2Teams
WHERE 
    rnk <= 2;

num_affected_rows,num_inserted_rows


In [0]:
%sql
--Best four teams from each group which are third best team
insert into round_of_16
SELECT *
FROM (
    SELECT 
        TeamId,
        GroupName,
        points,
        goaldifference,
        ROW_NUMBER() OVER (PARTITION BY GroupName ORDER BY points DESC, goaldifference desc,goalsScored desc) AS rnk 
    FROM 
        matchStats 
) AS Top2Teams
WHERE 
    rnk = 3 LIMIT 4;

num_affected_rows,num_inserted_rows
4,4


## Inserting the Round of 16 matches into the matches table

In [0]:
%sql
--Top Four Team Combinations
INSERT INTO matches (MatchId, fixture, GroupName,HomeTeam,AwayTeam, HomeTeamGoals, AwayTeamGoals, MatchResult, MatchStage)
WITH Combination AS (
  SELECT 
        CONCAT_WS('', sort_array( Collect_list(GroupName))) AS GroupCombination
  FROM round_of_16
  WHERE rnk = 3 
),

keys as(
    select * from thirdplacecombination 
        where selectedgroups = (select GroupCombination from Combination))
                        
SELECT 
    Row_Number() over(order by 1) + 36 as MatchId,
    CONCAT(r1.teamid, ' Vs. ', r2.teamid) AS fixture,
    'N' as GroupName,
    r1.TEAMID as HomeTeam,
    r2.TEAMID as AwayTeam,
      CAST(rand() * 6 AS INT) AS GoalsScored, 
      CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' as MatchResult,
    'R16' as MatchStage
    
FROM 
    round_of_16 r1
JOIN 
    round_of_16 r2 ON(
     (r1.groupname = 'B' AND r1.rnk = 1 AND r2.groupname = (select Team1B from keys) AND r2.rnk = 3) OR
      (r1.groupname = 'A' AND r1.rnk = 1 AND r2.groupname = 'C' AND r2.rnk = 2) OR
      (r1.groupname = 'F' AND r1.rnk = 1 AND r2.groupname = (select Team1F from keys) AND r2.rnk = 3) OR
      (r1.groupname = 'D' AND r1.rnk = 2 AND r2.groupname = 'E' AND r2.rnk = 2) OR
      (r1.groupname = 'E' AND r1.rnk = 1 AND r2.groupname = (select Team1E from keys) AND r2.rnk = 3) OR
      (r1.groupname = 'D' AND r1.rnk = 1 AND r2.groupname = 'F' AND r2.rnk = 2) OR
      (r1.groupname = 'C' AND r1.rnk = 1 AND r2.groupname = (select Team1C from keys) AND r2.rnk = 3) OR
      (r1.groupname = 'A' AND r1.rnk = 2 AND r2.groupname = 'B' AND r2.rnk = 2)
      );


num_affected_rows,num_inserted_rows
8,8


In [0]:
%sql
UPDATE Matches
SET MatchResult = 
    CASE
        WHEN HOMETEAMGOALS > AWAYTEAMGOALS THEN HomeTeam
        WHEN HOMETEAMGOALS < AWAYTEAMGOALS THEN AwayTeam
         ELSE 
            CASE 
                WHEN (HomeTeamGoals + 1) % 2 = 1 THEN HomeTeam
                ELSE AwayTeam
            END
    END
  WHERE MatchStage = "R16";

num_affected_rows
8


## Matches for the Quater-Final Round 

In [0]:
%sql

INSERT INTO matches (MatchId, fixture,GroupName, HomeTeam, AwayTeam, HomeTeamGoals, AwayTeamGoals, MatchResult, MatchStage)
SELECT 
    Row_Number() OVER(ORDER BY 1) + 44 AS MatchId,
    CONCAT(QF1.MATCHRESULT, ' Vs. ', QF2.MATCHRESULT) AS fixture,
    'N' as GroupName,
    QF1.MATCHRESULT AS HomeTeam,
    QF2.MATCHRESULT AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'Quarter-Final' AS MatchStage
FROM 
    matches QF1
JOIN 
    matches QF2 ON (
        (QF1.MATCHID = 39 AND QF2.MatchId = 37) OR
        (QF1.MATCHID = 41 AND QF2.MatchId = 42) OR
        (QF1.MATCHID = 43 AND QF2.MatchId = 44) OR
        (QF1.MATCHID = 40 AND QF2.MatchId = 38)
    );

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
UPDATE Matches
SET MatchResult = 
    CASE
        WHEN HOMETEAMGOALS > AWAYTEAMGOALS THEN HomeTeam
        WHEN HOMETEAMGOALS < AWAYTEAMGOALS THEN AwayTeam
        ELSE 
        CASE 
                WHEN (HomeTeamGoals + 1) % 2 = 1 THEN HomeTeam
                ELSE AwayTeam
            END
    END
WHERE MatchStage = 'Quarter-Final';

num_affected_rows
4


## Matches For the Semi-Final Round

In [0]:
%sql

INSERT INTO matches (MatchId, fixture,GroupName, HomeTeam, AwayTeam, HomeTeamGoals, AwayTeamGoals, MatchResult, MatchStage)
SELECT 
    Row_Number() OVER(ORDER BY 1) + 48 AS MatchId,
    CONCAT(SF1.MATCHRESULT, ' Vs. ', SF2.MATCHRESULT) AS fixture,
    'N' as GroupName,
    SF1.MATCHRESULT AS HomeTeam,
    SF2.MATCHRESULT AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'Semi-Final' AS MatchStage
FROM 
    matches SF1
JOIN 
    matches SF2 ON (
        (SF1.MATCHID = 45 AND SF2.MatchId = 46) OR
        (SF1.MATCHID = 47 AND SF2.MatchId = 48)
    );

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
UPDATE Matches
SET MatchResult = 
    CASE
        WHEN HOMETEAMGOALS > AWAYTEAMGOALS THEN HomeTeam
        WHEN HOMETEAMGOALS < AWAYTEAMGOALS THEN AwayTeam
        ELSE 
          CASE 
                  WHEN (HomeTeamGoals + 1) % 2 = 1 THEN HomeTeam
                  ELSE AwayTeam
              END
    END
WHERE MatchStage = 'Semi-Final';

num_affected_rows
2


## Match For Third Place

In [0]:
%sql
INSERT INTO matches (MatchId, fixture,GroupName, HomeTeam, AwayTeam, HomeTeamGoals, AwayTeamGoals, MatchResult, MatchStage)
WITH LosingTeams AS (
    SELECT
        CASE
            WHEN MATCHRESULT != HOMETEAM THEN HOMETEAM
            ELSE AWAYTEAM
        END AS LosingTeam
    FROM
        matches
    WHERE 
        MATCHSTAGE = 'Semi-Final'
)

SELECT 
    Row_Number() OVER(ORDER BY 1) + 50 AS MatchId,
    CONCAT(T1.LosingTeam, ' Vs. ', T2.LosingTeam) AS fixture,
    'N' as GroupName,
    T1.LosingTeam AS HomeTeam,
    T2.LosingTeam AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'Third-Place' AS MatchStage
FROM 
    LosingTeams T1
JOIN 
    LosingTeams T2 ON T1.LosingTeam < T2.LosingTeam;


num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
UPDATE Matches
SET MatchResult = 
    CASE
        WHEN HOMETEAMGOALS > AWAYTEAMGOALS THEN HomeTeam
        WHEN HOMETEAMGOALS < AWAYTEAMGOALS THEN AwayTeam
        ELSE 
          CASE 
                  WHEN (HomeTeamGoals + 1) % 2 = 1 THEN HomeTeam
                  ELSE AwayTeam
              END
    END
WHERE MatchStage = 'Third-Place';

num_affected_rows
1


## Match For the Final

In [0]:
%sql
INSERT INTO matches (MatchId, fixture,GroupName, HomeTeam, AwayTeam, HomeTeamGoals, AwayTeamGoals, MatchResult, MatchStage)
SELECT 
    Row_Number() OVER(ORDER BY 1) + 52 AS MatchId,
    CONCAT(F1.MATCHRESULT, ' Vs. ', F2.MATCHRESULT) AS fixture,
    'N' as GroupName,
    F1.MATCHRESULT AS HomeTeam,
    F2.MATCHRESULT AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'Final' AS MatchStage
FROM 
    matches F1
JOIN 
    matches F2 ON 
        F1.MATCHID = 49 AND F2.MatchId = 50;

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
UPDATE Matches
SET MatchResult = 
    CASE
        WHEN HOMETEAMGOALS > AWAYTEAMGOALS THEN HomeTeam
        WHEN HOMETEAMGOALS < AWAYTEAMGOALS THEN AwayTeam
        ELSE 
          CASE 
                WHEN (HomeTeamGoals + 1) % 2 = 1 THEN HomeTeam
                ELSE AwayTeam
            END
    END
WHERE MatchStage = 'Final';

num_affected_rows
1


In [0]:
%sql
select * from matches where MatchStage = 'R16' order by MatchId;

MatchID,Fixture,GroupName,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,MatchResult,MatchStage
37,Switzerland Vs. Serbia,N,Switzerland,Serbia,1,1,Serbia,R16
38,Hungary Vs. Spain,N,Hungary,Spain,0,2,Spain,R16
39,Albania Vs. Germany,N,Albania,Germany,0,0,Albania,R16
40,Slovenia Vs. Austria,N,Slovenia,Austria,4,5,Austria,R16
41,France Vs. Czech Republic,N,France,Czech Republic,5,2,France,R16
42,Netherlands Vs. Play-off Winner B,N,Netherlands,Play-off Winner B,4,1,Netherlands,R16
43,Slovakia Vs. Italy,N,Slovakia,Italy,2,4,Italy,R16
44,Play-off Winner C Vs. Denmark,N,Play-off Winner C,Denmark,0,1,Denmark,R16


In [0]:
%sql
SELECT * FROM Matches where MatchStage = 'Quarter-Final' order by MatchId;

MatchID,Fixture,GroupName,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,MatchResult,MatchStage
45,Albania Vs. Serbia,N,Albania,Serbia,1,5,Serbia,Quarter-Final
46,Austria Vs. Spain,N,Austria,Spain,1,4,Spain,Quarter-Final
47,France Vs. Netherlands,N,France,Netherlands,3,2,France,Quarter-Final
48,Italy Vs. Denmark,N,Italy,Denmark,3,5,Denmark,Quarter-Final


In [0]:
%sql
SELECT * FROM Matches where MatchStage = 'Semi-Final' order by MatchId;

MatchID,Fixture,GroupName,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,MatchResult,MatchStage
49,Serbia Vs. Spain,N,Serbia,Spain,2,5,Spain,Semi-Final
50,France Vs. Denmark,N,France,Denmark,1,2,Denmark,Semi-Final


In [0]:
%sql
SELECT * FROM Matches where MatchStage = 'Third-Place' order by MatchId;

MatchID,Fixture,GroupName,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,MatchResult,MatchStage
51,France Vs. Serbia,N,France,Serbia,1,5,Serbia,Third-Place


In [0]:
%sql
SELECT * FROM Matches where MatchStage = 'Final' order by MatchId;

MatchID,Fixture,GroupName,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,MatchResult,MatchStage
53,Spain Vs. Denmark,N,Spain,Denmark,3,5,Denmark,Final


# Matches Visualization For Each Stage

In [0]:
%python
import pandas as pd
from pyspark.sql import SparkSession


In [0]:
%python
#Spark Session
spark = SparkSession.builder \
    .appName("euro_cup_visulization") \
    .getOrCreate()

In [0]:
#Query from spark sql
round_of_16 = spark.sql("SELECT TeamId FROM Matches WHERE MatchStage = 'R16' ORDER BY MatchId")
quater_final = spark.sql("SELECT TeamId  FROM Matches WHERE stage = 'Quarter Final' ORDER BY MatchId")
semi_final = spark.sql("SELECT TeamId  FROM Matches WHERE stage = 'Semi-Final' ORDER BY MatchId")
final = spark.sql("SELECT TeamId  FROM Matches WHERE stage = 'Final' ORDER BY MatchId")
third_place = spark.sql("SELECT TeamId  FROM Matches WHERE stage = 'Third_Place' ORDER BY MatchId")


<command-4294186947500829>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  visualize_table16 = visualize_table16.append({'Roundof16': ''}, ignore_index=True)
<command-4294186947500829>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  visualize_table16 = visualize_table16.append({'Roundof16': ''}, ignore_index=True)
<command-4294186947500829>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  visualize_table16 = visualize_table16.append({'Roundof16': f"{row['Team1']} ({row['Team1Score']})"}, ignore_index=True)
<command-4294186947500829>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  visualize_table16 = visualize_table16.append({'Roundof

In [0]:
#Convert Spark DataFrame to Pandas Dataframe
round_of_16_df = round_of_16.toPandas()
quarter_final_df = quarter_final.toPandas()
semi_final_df = semi_final.toPandas()
final_df = final.toPandas()
third_place_df = third_place.toPandas()

,Roundof16,space
0,,
1,,
2,Ukraine (3),
3,Austria (4),
4,,
5,,
6,Netherlands (2),
7,Türkiye (5),
8,,
9,,


In [0]:
def create_empty_rows(num_rows):
    return pd.DataFrame(['']* num_rows)

In [0]:
visualize_r16 = pd.DataFrame({
    'Round_of_16': round_of_16_df['TeamId']
})
visualize_r16 = visualize_16.append(create_empty_rows(len(visualize_r16)*2), ignore_index =True)

In [0]:
euro_cup_df = pd.concat([visualize_r16],axix = 1)

In [0]:
display(euro_cup_df)